In [5]:
!pip install pandas yfinance statsmodels sympy

In [6]:
import statsmodels.api as sm
import yfinance as yf
import pandas as pd
from sympy import symbols, Eq, solve

# Define the function to load CSV data
def load_csv_with_date_index_v2(filepath, skiprows):
    """
    Load a CSV file into a pandas DataFrame, skipping a specified number of initial rows,
    and set the date column as the index.

    :param filepath: Path to the CSV file.
    :param skiprows: Number of rows to skip at the start of the file.
    :return: pandas DataFrame containing the data from the CSV file, with date as index.
    """
    import pandas as pd

    # Load the CSV file, skipping the specified number of rows
    df = pd.read_csv(filepath, skiprows=skiprows)

    # Explicitly converting the first column to string and then strip spaces
    df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.strip()
    df['Date'] = pd.to_datetime(df.iloc[:, 0], format='%Y%m', errors='coerce')

    # Drop the original first column and set 'Date' as index
    df = df.drop(df.columns[0], axis=1)
    df.set_index('Date', inplace=True)

    return df
X_rf, X_stock1, X_stock2, X_stock3 = symbols('X_rf X_stock1 X_stock2 X_stock3')

# Define the function to create a tracking portfolio
def create_tracking_portfolio(target_betas, stock_betas, rf_rate):
    # Symbols for the weights of the tracking portfolio
    global X_rf, X_stock1, X_stock2, X_stock3

    # Equations to solve for the weights based on target stock's factor exposures
    eq1 = Eq(X_rf + X_stock1 + X_stock2 + X_stock3, 1)  # Weights sum to 1
    eq2 = Eq(sum(X_stock * stock_betas[i]['Mkt-RF'] for i, X_stock in enumerate([X_stock1, X_stock2, X_stock3])), target_betas['Mkt-RF'])
    eq3 = Eq(sum(X_stock * stock_betas[i]['SMB'] for i, X_stock in enumerate([X_stock1, X_stock2, X_stock3])), target_betas['SMB'])
    eq4 = Eq(sum(X_stock * stock_betas[i]['HML'] for i, X_stock in enumerate([X_stock1, X_stock2, X_stock3])), target_betas['HML'])

    # Solve the system of equations
    solution = solve((eq1, eq2, eq3, eq4), (X_rf, X_stock1, X_stock2, X_stock3), dict=True)

    # If a solution exists, return it
    if solution:
        return solution[0]
    else:
        # No solution found
        return None

# Example usage
tickers = ['AAPL', 'MSFT', 'DIS', 'GOOG']  # Include a fourth stock (e.g., GOOG)
stock_to_track = 'AAPL'
other_stocks = [ticker for ticker in tickers if ticker != stock_to_track]

# Fetch historical stock data
stock_data = yf.download(tickers, start="2020-08-01", end="2023-07-31", interval='1mo')['Adj Close']

# Calculate monthly returns for each stock
monthly_returns = stock_data.pct_change().dropna()

# Load the Fama-French factor data
factor_data = load_csv_with_date_index_v2('Developed_3_Factors 2.csv', 6)

# Align the stock returns with the factor data
aligned_data = monthly_returns.join(factor_data, how='inner')

# Perform regression for each stock against the Fama-French factors
regression_results = {}
for ticker in tickers:
    X = sm.add_constant(aligned_data[['Mkt-RF', 'SMB', 'HML']]) # Fama-French factors
    y = aligned_data[ticker] # Stock returns
    model = sm.OLS(y, X).fit()
    regression_results[ticker] = model.params
# Calculate expected returns for each stock
expected_returns = {}
for ticker in tickers:
    betas = regression_results[ticker]
    # Calculate expected return for each period and then take an average
    expected_return_for_periods = factor_data.apply(
        lambda x: x['RF'] + betas['Mkt-RF'] * x['Mkt-RF'] + betas['SMB'] * x['SMB'] + betas['HML'] * x['HML'],
        axis=1
    )
    expected_returns[ticker] = expected_return_for_periods.mean()

# Print the expected returns

# For demonstration, let's just print the beta values for each stock

# Construct the tracking portfolio
stock_betas = [regression_results[ticker] for ticker in other_stocks]  # List of betas for the other stocks
target_betas = regression_results[stock_to_track]  # Betas for the stock to track
rf_rate = factor_data['RF'].iloc[-1]  # Most recent risk

# Construct the tracking portfolio for the stock we want to track
target_betas = regression_results[stock_to_track]  # Betas for the stock to track
portfolio_betas = [regression_results[ticker] for ticker in other_stocks]  # List of betas for the other stocks
rf_rate = factor_data['RF'].iloc[-1]  # Most recent risk-free rate

# Solve for the weights
tracking_portfolio_weights = create_tracking_portfolio(target_betas, portfolio_betas, rf_rate)

# Check if the solution was found
if tracking_portfolio_weights:
    # Calculate the expected return of the tracking portfolio using the weights
    tracking_portfolio_return = (
        tracking_portfolio_weights[X_rf] * rf_rate +
        tracking_portfolio_weights[X_stock1] * expected_returns[other_stocks[0]] +
        tracking_portfolio_weights[X_stock2] * expected_returns[other_stocks[1]] +
        tracking_portfolio_weights[X_stock3] * expected_returns[other_stocks[2]]
    )

    # Calculate the alpha (arbitrage opportunity)
    target_expected_return = expected_returns[stock_to_track]
    alpha = tracking_portfolio_return - target_expected_return

    # Determine if there's an arbitrage opportunity
    arbitrage_opportunity = alpha > 0
    arbitrage_guidance = "Go long in the tracking portfolio and short since the expected return for the tracking portfolio is larger than the expected return of the target stock" + stock_to_track if arbitrage_opportunity else "No arbitrage opportunity"
else:
    print("The system of equations did not return a valid solution.")
for ticker, exp_return in expected_returns.items():
    print(f"Expected return for {ticker} based on Arbitrage Pricing Theory: {exp_return}")

# Output the results
print(f"Tracking Portfolio Weights: {tracking_portfolio_weights if tracking_portfolio_weights else 'No valid solution found.'}")

print(f"Expected return for the Tracking Portfolio: {tracking_portfolio_return if 'tracking_portfolio_return' in locals() else 'N/A'}")
print(f"Alpha: {alpha if 'alpha' in locals() else 'N/A'}")
print(f"Arbitrage Guidance: {arbitrage_guidance if 'arbitrage_guidance' in locals() else 'N/A'}")

[*********************100%%**********************]  4 of 4 completed
Expected return for AAPL based on Arbitrage Pricing Theory: 0.20937925291967627
Expected return for MSFT based on Arbitrage Pricing Theory: 0.20830642406625077
Expected return for DIS based on Arbitrage Pricing Theory: 0.21400798874333446
Expected return for GOOG based on Arbitrage Pricing Theory: 0.20962547707987117
Tracking Portfolio Weights: {X_rf: 0.0348235555642949, X_stock1: -2.82517388031258, X_stock2: -0.875627894841538, X_stock3: 4.66597821958983}
Expected return for the Tracking Portfolio: 0.217188806182566
Alpha: 0.00780955326289015
Arbitrage Guidance: Go long in the tracking portfolio and short since the expected return for the tracking portfolio is larger than the expected return of the target stockAAPL
